# Entrenamiento personalizado de YOLO

In [ ]:
import os

print("Vamos a cambiar el directorio de trabajo")

# Indicamos la ruta del directorio de trabajo
route = os.getcwd()+"/TFG/test/PNe_segmentation"
os.chdir(route)

current_directory = os.getcwd()
print(" El directorio actual es:", current_directory)

# Listamos el contenido del directorio
files = os.listdir(current_directory)
print(" Contenido del directorio actual:")
for file in files:
    print("\t",file)
    
# Listamos el contenido del directorio de las máscaras
# masks_directory = route+"TFG\\test\\PNe_segmentation\\masks"
# data_directory = route+"TFG\\test\\PNe_segmentation\\data"
## Ejecución en el CESGA Finisterrae III
masks_directory = current_directory+"/masks"
data_directory = current_directory+"/data"

Inspeccionamos el modelo from scratch para saber la entrada y la salida que espera

In [ ]:
from ultralytics import YOLO
import torch

# Create a new YOLO model from scratch
model = YOLO("yolov8n-seg.yaml", task='segment', verbose=True)

In [ ]:
model

Modificamos el modelo para que en vez de recibir 3 capas de entrada reciba uno solo

In [ ]:
model.model.model[0]

In [ ]:
# Acceder a la capa de convolución inicial
conv_layer = model.model.model[0].conv

# Crear una nueva capa de convolución con 1 canal de entrada en lugar de 3
new_conv_layer = torch.nn.Conv2d(1, conv_layer.out_channels, kernel_size=conv_layer.kernel_size, stride=conv_layer.stride, padding=conv_layer.padding, bias=conv_layer.bias is not None)

# Reemplazar la capa de convolución en el modelo
model.model.model[0].conv = new_conv_layer

# Verificar la nueva configuración de la capa
print(model.model.model[0])

# # Acceder a la capa de convolución inicial
# conv_layer = model.model.model[0].conv
# bn_layer = model.model.model[0].bn
# act_layer = model.model.model[0].act

# # Crear una nueva capa de convolución con 1 canal de entrada en lugar de 3
# new_conv_layer = torch.nn.Conv2d(1, 3, kernel_size=1, padding='same', bias=False)

# new_first_block = torch.nn.Sequential(new_conv_layer, conv_layer, bn_layer, act_layer)

# # Reemplazar la capa de convolución en el modelo
# model.model.model[0] = new_first_block

# # Verificar la nueva configuración de la capa
# print(model.model.model[0])

In [ ]:
from torch import nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(32, 1, kernel_size=1, stride=1, padding=0),
            nn.Upsample(scale_factor=2),
            nn.Upsample(scale_factor=2)
        )
        self.f = -1
        self.i = 0
    
    def forward(self, x):
        return self.model(x)
    
    def get_f(self):
        return self.f
    
    def set_f(self, f):
        self.f = f
        
model.model.model.append(MyModel())

Ahora que ya tenemos el modelo modificado, vamos a probar a cargarlo con nuestro Lightning Module personalizado para entrenarlo a nuestro manera

In [ ]:
from pnebulae_torch.models import smpAdapter
import segmentation_models_pytorch as smp


model = model.model
conv_model = smpAdapter(model = model, learning_rate=0.0001, threshold=0.5, current_fold=0, loss_fn=smp.losses.DiceLoss, scheduler=None)

In [ ]:
from pnebulae_torch.dataset import NebulaeDataset
from pnebulae_torch.preprocess import CutValues
from pnebulae_torch.normalize import TypicalImageNorm
from torchvision import transforms


import pandas as pd

transform_x = transforms.Compose([
                    # MinMaxNorm,
                    CutValues(factor = 2),
                    TypicalImageNorm(factor = 1, substract=0),
                    # MinMaxImageNorm(min = -88.9933, max=125873.7500),
                    # ApplyMorphology(operation = morphology.binary_opening, concat = True, footprint = morphology.disk(2)),
                    # ApplyMorphology(operation = morphology.area_opening, concat = True, area_threshold = 200, connectivity = 1),
                    # ApplyIntensityTransformation(transformation = exposure.equalize_hist, concat = True, nbins = 4096),
                    # ApplyIntensityTransformation(transformation = exposure.equalize_adapthist, concat = True, nbins = 640, kernel_size = 5),
                    # ApplyMorphology(operation = morphology.area_opening, concat = True, area_threshold = 200, connectivity = 1),
                    # ApplyFilter(filter = ndimage.gaussian_filter, concat = True, sigma = 5),
                    # transforms.ToTensor(),
                    # CustomPad(target_size = (1984, 1984), fill_min=True, tensor_type=torch.Tensor.float)
                    # ApplyIntensityTransformation(transformation = exposure.equalize_hist, concat = False, nbins = 256),
                    transforms.ToTensor(),
                    ])
type_fnc = torch.Tensor.int
transform_y = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Lambda(lambda x: type_fnc(x.round())),
                    # CustomPad(target_size = (1984, 1984), fill = 0, tensor_type=torch.Tensor.int)
                    ])

df_train = pd.read_csv("data_files_1c_train.csv")
dataset_train = NebulaeDataset(data_directory, masks_directory, df_train, transform = (transform_x, transform_y))

In [ ]:
conv_model.train(True)

In [ ]:
a = conv_model(dataset_train[156][0].unsqueeze(0))

In [ ]:
a.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(torch.sigmoid(a)[0][0].detach().numpy() > 0.55, cmap='gray')

Vamos a intentar hacer un entrenamiento al igual que hacemos para el resto de modelos

In [ ]:
from pnebulae_torch.dataset import NebulaeDataset
from pnebulae_torch.preprocess import ApplyMorphology, ApplyIntensityTransformation, ApplyFilter, CustomPad, CutValues
from pnebulae_torch.normalize import TypicalImageNorm, MinMaxImageNorm
from pnebulae_torch.models.callbacks import PrintCallback
from pnebulae_torch.models import basicUNet, smpAdapter, ConvNet
from pnebulae_torch.utils import DivideWindowsSubset
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from sklearn.model_selection import KFold
from torchvision import transforms
from skimage import morphology, exposure
from scipy import ndimage
from lightning.pytorch import seed_everything
from segmentation_models_pytorch.losses import DiceLoss
from lightning.pytorch.loggers import WandbLogger
import segmentation_models_pytorch as smp
import torch
import os
import pandas as pd
import lightning as L
import wandb
import inspect
import time
import gc

if __name__ == "__main__":
    ########## CONFIGURACIÓN SCRIPT ##########
    # Establecemos la clave de la API de W&B
    os.environ["WANDB_API_KEY"] = "21924e6e134841c5c16842c4ac42fcbe5a66feb2"
    ruta_logs_wandb = os.environ["STORE"] + "/TFG/logs_wandb/"
    
    torch.set_float32_matmul_precision('high')
    
    ####### CONFIGURACIÓN ENTRENAMIENTO #######
    model_name = "YOLO_test"
    
    BATCH_SIZE = 124
    num_epochs = 2000
    lr = 1e-2
    window_shape = 640
    
    k = 5
    
    loss_fn = DiceLoss
    activation_layer=torch.nn.ReLU
    
    if "mode" in inspect.signature(loss_fn).parameters:
        type_fnc = torch.Tensor.int
    else:
        type_fnc = torch.Tensor.float
        
    ############# CARGA DATASET #############
    transform_x = transforms.Compose([
                        # MinMaxNorm,
                        CutValues(factor = 2),
                        TypicalImageNorm(factor = 1, substract=0),
                        # MinMaxImageNorm(min = -88.9933, max=125873.7500),
                        # ApplyMorphology(operation = morphology.binary_opening, concat = True, footprint = morphology.disk(2)),
                        # ApplyMorphology(operation = morphology.area_opening, concat = True, area_threshold = 200, connectivity = 1),
                        # ApplyIntensityTransformation(transformation = exposure.equalize_hist, concat = True, nbins = 4096),
                        # ApplyIntensityTransformation(transformation = exposure.equalize_adapthist, concat = True, nbins = 640, kernel_size = 5),
                        # ApplyMorphology(operation = morphology.area_opening, concat = True, area_threshold = 200, connectivity = 1),
                        # ApplyFilter(filter = ndimage.gaussian_filter, concat = True, sigma = 5),
                        # transforms.ToTensor(),
                        # CustomPad(target_size = (1984, 1984), fill_min=True, tensor_type=torch.Tensor.float)
                        # ApplyIntensityTransformation(transformation = exposure.equalize_hist, concat = False, nbins = 256),
                        transforms.ToTensor(),
                        ])

    transform_y = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Lambda(lambda x: type_fnc(x.round())),
                        # CustomPad(target_size = (1984, 1984), fill = 0, tensor_type=torch.Tensor.int)
                        ])

    df_train = pd.read_csv("data_files_1c_train.csv")
    dataset_train = NebulaeDataset(data_directory, masks_directory, df_train, transform = (transform_x, transform_y))
    
    df_test = pd.read_csv("data_files_1c_test.csv")
    dataset_test = NebulaeDataset(data_directory, masks_directory, df_test, transform = (transform_x, transform_y))

    seed_everything(42, workers = True)
    
    ########## ENTRENAMIENTO MODELO ##########
    # Definimos el K-fold Cross Validator
    kfold = KFold(n_splits=k, shuffle=True, random_state = 42)
    
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset_train)):
        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath=os.environ["STORE"] + f"/TFG/model_checkpoints/{model_name}",
            filename='best_model-{epoch:02d}-'+str(fold),
            save_top_k=1,
            mode='min',
        )
        
        checkpoint_callback_last = ModelCheckpoint(
            monitor=None,
            dirpath=os.environ["STORE"] + f"/TFG/model_checkpoints/{model_name}",
            filename='last_model_fold'+str(fold),
        )
        
        callbacks = [PrintCallback(), LearningRateMonitor(logging_interval='epoch'), checkpoint_callback, checkpoint_callback_last]
        
        # Acceder a la capa de convolución inicial
        model = YOLO("yolov8n-seg.yaml", task='segment')
        
        conv_layer = model.model.model[0].conv

        # Crear una nueva capa de convolución con 1 canal de entrada en lugar de 3
        new_conv_layer = torch.nn.Conv2d(dataset_train[0][0].shape[0], conv_layer.out_channels, kernel_size=conv_layer.kernel_size, stride=conv_layer.stride, padding=conv_layer.padding, bias=conv_layer.bias is not None)

        # Reemplazar la capa de convolución en el modelo
        model.model.model[0].conv = new_conv_layer

        class MyModel(nn.Module):
            def __init__(self):
                super(MyModel, self).__init__()
                self.model = torch.nn.Sequential(
                    torch.nn.Conv2d(32, 1, kernel_size=1, stride=1, padding=0),
                    torch.nn.Upsample(scale_factor=2),
                    torch.nn.Upsample(scale_factor=2)
                )
                self.f = -1
                self.i = 0
            
            def forward(self, x):
                return self.model(x)
            
            def get_f(self):
                return self.f
            
            def set_f(self, f):
                self.f = f
                
        model.model.model.append(MyModel())
        # # Acceder a la capa de convolución inicial
        # conv_layer = model.model.model[0].conv
        # bn_layer = model.model.model[0].bn
        # act_layer = model.model.model[0].act

        # # Crear una nueva capa de convolución con 1 canal de entrada en lugar de 3
        # new_conv_layer = torch.nn.Conv2d(1, 3, kernel_size=1, padding='same', bias=False)

        # new_first_block = torch.nn.Sequential(new_conv_layer, conv_layer, bn_layer, act_layer)

        # # Reemplazar la capa de convolución en el modelo
        # model.model.model[0] = new_first_block
        
        model = model.model
        # Definimos el modelo con los pesos inicializados aleatoriamente (sin preentrenar)
        # model = smpAdapter(model = model, learning_rate=lr, threshold=0.5, current_fold=fold, loss_fn=loss_fn, scheduler=None)
        # model = smpAdapter(model = model, learning_rate=lr, threshold=0.5, current_fold=fold, loss_fn=loss_fn, scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau, mode='min', factor=0.1, patience=500, cooldown=150, verbose=False)
        model = smpAdapter(model = model, learning_rate=lr, threshold=0.5, current_fold=fold, loss_fn=loss_fn, scheduler=torch.optim.lr_scheduler.StepLR, step_size = 500, gamma = 0.1, verbose=False)
        # model = smpAdapter(model = model, learning_rate=lr, threshold=0.5, current_fold=fold, loss_fn=loss_fn, scheduler=torch.optim.lr_scheduler.MultiStepLR, milestones = [1000, 4000], gamma = 0.1, verbose=False)
        
        ruta_logs_wandb = os.environ["STORE"] + "/TFG/logs_wandb/"
        logger_wandb = WandbLogger(project="segmentation_TFG", log_model = False, name=model_name, save_dir=ruta_logs_wandb)
        logger_wandb.experiment.config.update({"model_name": model_name})

        # log gradients, parameter histogram and model topology
        logger_wandb.watch(model, log="all")

        trainer = L.Trainer(strategy='auto', max_epochs=num_epochs, accelerator='cuda', log_every_n_steps=2, logger= logger_wandb, callbacks=callbacks)

        # Imprimimos el fold del que van a mostrarse los resultados
        print('--------------------------------')
        print(f"Model info:\n\t- Batch Size: {BATCH_SIZE}\n\t- GPUs on use: {torch.cuda.device_count()}")

        # Creamos nuestros propios Subsets de PyTorch aplicando a cada conjunto la transformacion deseada
        train_subset = torch.utils.data.Subset(dataset_train, train_ids)
        val_subset = torch.utils.data.Subset(dataset_train, val_ids)
        
        if window_shape is not None:
            train_subset = DivideWindowsSubset(train_subset, window_shape = window_shape, fill_min = True)
            val_subset = DivideWindowsSubset(val_subset, window_shape = window_shape, fill_min = True)
        
        # Definimos un data loader por cada conjunto de datos que vamos a utilizar.
        trainloader = torch.utils.data.DataLoader(
                                train_subset,
                                batch_size=BATCH_SIZE, num_workers=6, shuffle=True, persistent_workers=False)

        valloader = torch.utils.data.DataLoader(
                                val_subset,
                                batch_size=BATCH_SIZE, num_workers=6, shuffle=False, persistent_workers=False)
        
        # Entrenamos el modelo, extrayendo los resultados y guardandolos en la variable result, y evaluamos en el conjunto de test.
        trainer.fit(model, trainloader, valloader) 

        logger_wandb.experiment.unwatch(model)

        # testloader = torch.utils.data.DataLoader(
        #                         dataset_test,
        #                         batch_size=BATCH_SIZE, num_workers=8, shuffle=False, persistent_workers=True)
        
        # Creamos un nuevo entrenador con una sola GPU para la fase de prueba
        # trainer_test = L.Trainer(devices = 1, strategy='auto', max_epochs=num_epochs, accelerator='cuda', log_every_n_steps=1, logger=logger_wandb, callbacks=callbacks)
        # trainer_test.test(model, testloader)

        logger_wandb.finalize("success")
        wandb.finish()
        
        del model
        del trainer
        
        torch.cuda.empty_cache()
        time.sleep(30)

# Entrenamiento por defecto de YOLO

Hasta este punto hemos conseguido entrenar el modelo YOLO, con una cabeza final personalizada y empleando nuestro método de entrenamiento, lo que vamos a proabr ahora es a entrenar YOLO tal y como viene preparado en la librería, pero vamos a seguir intentando hacer el 5Fold-CrossValidation e introduciendo las mismas imágenes que hemos probado para el resto de modelos. Para ellos lo que vamos a hacer es guardar todas las imágenes, en las ventanas de 512 respectivas y para cada fold las vamos a ir moviendo entre las diferentes carpetas.


In [1]:
import os
os.chdir(os.environ['HOME'])

In [2]:
import os

print("Vamos a cambiar el directorio de trabajo")

# Indicamos la ruta del directorio de trabajo
route = os.getcwd()+"/TFG/test/PNe_segmentation"
os.chdir(route)

current_directory = os.getcwd()
print(" El directorio actual es:", current_directory)

# Listamos el contenido del directorio
files = os.listdir(current_directory)
print(" Contenido del directorio actual:")
for file in files:
    print("\t",file)
    
# Listamos el contenido del directorio de las máscaras
# masks_directory = route+"TFG\\test\\PNe_segmentation\\masks"
# data_directory = route+"TFG\\test\\PNe_segmentation\\data"
## Ejecución en el CESGA Finisterrae III
masks_directory = current_directory+"/masks"
data_directory = current_directory+"/data"

os.chdir(route+"/../yolo_segmentation")
save_directory = os.getcwd()

Vamos a cambiar el directorio de trabajo
 El directorio actual es: /mnt/netapp2/Home_FT2/home/ulc/co/ela/TFG/test/PNe_segmentation
 Contenido del directorio actual:
	 train_models
	 data
	 create_dataset.ipynb
	 data_files_1c.csv
	 images
	 image_analysis.ipynb
	 masks
	 segmentation_no_supervisada.ipynb
	 segmentation_no_supervisada_2c.ipynb
	 segmentation_no_supervisada_pytorch.ipynb
	 segmentation_supervisada.ipynb
	 dataset_info.csv
	 data_files_1c_train.csv
	 historico_notebooks
	 data_files_1c_test.csv
	 segmentation_no_supervisada_pytorch_clean.ipynb
	 segmentation_supervisada_pytorch copy.ipynb
	 segmentation_supervisada_pytorch.ipynb
	 cesga
	 segmentation_supervisada_pytorch copy 2.ipynb
	 segmentation_supervisada_pytorch copy 3.ipynb
	 data_files_1c_train_da.csv


## Convertir fits a JPG

Lo realizamos para el conjunto de entrenamiento

In [3]:
import numpy as np
from pnebulae_torch.dataset import NebulaeDataset
from pnebulae_torch.normalize import TypicalImageNorm
from pnebulae_torch.preprocess import CutValues
from pnebulae_torch.utils import DivideWindowsSubset
from skimage import exposure
from torchvision import transforms
import torch
import pandas as pd

transform_x = transforms.Compose([
                    CutValues(factor = 2),
                    TypicalImageNorm(factor = 1, substract=0),
                    transforms.ToTensor()
                    ])

transform_y = transforms.Compose([
                    transforms.ToTensor()
                    ])

df_train = pd.read_csv(route+"/data_files_1c_train.csv")
dataset_train = NebulaeDataset(data_directory, masks_directory, df_train, transform = (transform_x, transform_y))

train_subset = torch.utils.data.Subset(dataset_train, list(range(len(dataset_train))))
train_subset = DivideWindowsSubset(train_subset, window_shape = 512, fill_min = True)


In [4]:
from PIL import Image

if not os.path.exists(save_directory+"/segment_kfold/images/train"):
    os.makedirs(save_directory+"/segment_kfold/images/train")

for i in range(len(train_subset)):
    
    file_name = str(i+1).zfill(3)+".png"
    file_path = os.path.join(save_directory+"/segment_kfold/images/train", file_name)
    
    np_array = train_subset[i][0].permute(1, 2, 0).numpy()[:,:,0]
    
    image = Image.fromarray((np_array*255).round().astype(np.uint8))
    image.save(file_path)

In [5]:
import os
from skimage import measure
import numpy as np

if not os.path.exists(save_directory + "/segment_kfold/labels/train"):
    os.makedirs(save_directory + "/segment_kfold/labels/train")

if not os.path.exists(save_directory + "/segment_kfold/masks/train"):
    os.makedirs(save_directory + "/segment_kfold/masks/train")
    
for i in range(len(train_subset)):
    mask = train_subset[i][1].permute(1, 2, 0).numpy()[:,:,0]
    mask = (mask.round() > 0).astype(np.uint8)  # Asegurarse de que la máscara sea binaria
    
    mask_name = str(i+1).zfill(3)+".png"
    
    labeled_mask = measure.label(mask)
    regions = measure.regionprops(labeled_mask)

    height, width = mask.shape
    label_path = os.path.join(save_directory + "/segment_kfold/labels/train", mask_name.replace('.png', '.txt'))
    with open(label_path, 'w') as f:
        for region in regions:
            coords = region.coords.astype(np.float32)
            coords = coords[:, [1, 0]]  # Intercambiar columnas para tener (x, y)
            coords[:, 0] = coords[:, 0] / width  # Normalizar coordenadas x
            coords[:, 1] = coords[:, 1] / height  # Normalizar coordenadas y
            coords = coords.flatten()
            coords_str = ' '.join(map(str, coords))
            f.write(f"0 {coords_str}\n")
    
    mask_path = os.path.join(save_directory + "/segment_kfold/masks/train", mask_name)
    mask = Image.fromarray((mask*255).round().astype(np.uint8)).convert('L')
    mask.save(mask_path)
    

Y también para el de test

In [6]:
import numpy as np
from pnebulae_torch.dataset import NebulaeDataset
from pnebulae_torch.normalize import TypicalImageNorm
from pnebulae_torch.preprocess import CutValues
from pnebulae_torch.utils import DivideWindowsSubset
from skimage import exposure
from torchvision import transforms
import torch
import pandas as pd

transform_x = transforms.Compose([
                    CutValues(factor = 2),
                    TypicalImageNorm(factor = 1, substract=0),
                    transforms.ToTensor()
                    ])

transform_y = transforms.Compose([
                    transforms.ToTensor()
                    ])

df_test = pd.read_csv(route+"/data_files_1c_test.csv")
dataset_test = NebulaeDataset(data_directory, masks_directory, df_test, transform = (transform_x, transform_y))

test_subset = torch.utils.data.Subset(dataset_test, list(range(len(dataset_test))))
test_subset = DivideWindowsSubset(test_subset, window_shape = 512, fill_min = True)


In [7]:
from PIL import Image

if not os.path.exists(save_directory+"/segment_kfold/images/test"):
    os.makedirs(save_directory+"/segment_kfold/images/test")

for i in range(len(test_subset)):
    
    file_name = str(i+250).zfill(3)+".png"
    file_path = os.path.join(save_directory+"/segment_kfold/images/test", file_name)
    
    np_array = test_subset[i][0].permute(1, 2, 0).numpy()[:,:,0]
    
    image = Image.fromarray((np_array*255).round().astype(np.uint8))
    image.save(file_path)

In [8]:
import os
from skimage import measure
import numpy as np

if not os.path.exists(save_directory + "/segment_kfold/labels/test"):
    os.makedirs(save_directory + "/segment_kfold/labels/test")

if not os.path.exists(save_directory + "/segment_kfold/masks/test"):
    os.makedirs(save_directory + "/segment_kfold/masks/test")
    
for i in range(len(test_subset)):
    mask = test_subset[i][1].permute(1, 2, 0).numpy()[:,:,0]
    mask = (mask.round() > 0).astype(np.uint8)  # Asegurarse de que la máscara sea binaria
    
    mask_name = str(i+250).zfill(3)+".png"
    
    labeled_mask = measure.label(mask)
    regions = measure.regionprops(labeled_mask)

    height, width = mask.shape
    label_path = os.path.join(save_directory + "/segment_kfold/labels/test", mask_name.replace('.png', '.txt'))
    with open(label_path, 'w') as f:
        for region in regions:
            coords = region.coords.astype(np.float32)
            coords = coords[:, [1, 0]]  # Intercambiar columnas para tener (x, y)
            coords[:, 0] = coords[:, 0] / width  # Normalizar coordenadas x
            coords[:, 1] = coords[:, 1] / height  # Normalizar coordenadas y
            coords = coords.flatten()
            coords_str = ' '.join(map(str, coords))
            f.write(f"0 {coords_str}\n")
    
    mask_path = os.path.join(save_directory + "/segment_kfold/masks/test", mask_name)
    mask = Image.fromarray((mask*255).round().astype(np.uint8)).convert('L')
    mask.save(mask_path)
    

Ahora creamos las carpetas para validación si no existen y lo que se hará en cada fold será mover a estas carpetas, desde las carpetas train, todos los archivos que toquen en validación.

In [9]:
if not os.path.exists(save_directory + "/segment_kfold/images/val"):
    os.makedirs(save_directory + "/segment_kfold/images/val")
    
if not os.path.exists(save_directory + "/segment_kfold/labels/val"):
    os.makedirs(save_directory + "/segment_kfold/labels/val")

if not os.path.exists(save_directory + "/segment_kfold/masks/val"):
    os.makedirs(save_directory + "/segment_kfold/masks/val")

In [17]:
from ultralytics import YOLO
model = YOLO("yolov8n-seg.pt", task='segment')

Ahora realizamos el entrenamiento

In [10]:
from pnebulae_torch.dataset import NebulaeDataset
from pnebulae_torch.preprocess import  CutValues
from pnebulae_torch.normalize import TypicalImageNorm
from pnebulae_torch.utils import DivideWindowsSubset
from sklearn.model_selection import KFold
from torchvision import transforms
from lightning.pytorch import seed_everything
import torch
import os
import pandas as pd
import time
import shutil
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback

os.environ["WANDB_API_KEY"] = "21924e6e134841c5c16842c4ac42fcbe5a66feb2"
ruta_logs_wandb = os.environ["STORE"] + "/TFG/logs_wandb/"

torch.set_float32_matmul_precision('high')

####### CONFIGURACIÓN ENTRENAMIENTO #######
BATCH_SIZE = 124
num_epochs = 1000
lr = 1e-4
window_shape = 512

k = 5
    
############# CARGA DATASET #############
transform_x = transforms.Compose([
                    CutValues(factor = 2),
                    TypicalImageNorm(factor = 1, substract=0),
                    transforms.ToTensor(),
                    ])

transform_y = transforms.Compose([
                    transforms.ToTensor()
                    ])

df_train = pd.read_csv(route+"/data_files_1c_train.csv")
dataset_train = NebulaeDataset(data_directory, masks_directory, df_train, transform = (transform_x, transform_y))

seed_everything(42, workers = True)

########## ENTRENAMIENTO MODELO ##########
# Definimos el K-fold Cross Validator
kfold = KFold(n_splits=k, shuffle=True, random_state = 42)

for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset_train)):
    
    if fold != 1:
        continue
    
    cnt = 0
    for id in range(len(dataset_train)):
        subset = torch.utils.data.Subset(dataset_train, [id])
        subset = DivideWindowsSubset(subset, window_shape = window_shape, fill_min = True)
        
        if id in val_ids:
            for i in range(len(subset)):
                _ = shutil.move(save_directory + "/segment_kfold/images/train/" + str(id+1+i+cnt).zfill(3) + ".png", save_directory + "/segment_kfold/images/val/" + str(id+1+i+cnt).zfill(3) + ".png")
                _ = shutil.move(save_directory + "/segment_kfold/labels/train/" + str(id+1+i+cnt).zfill(3) + ".txt", save_directory + "/segment_kfold/labels/val/" + str(id+1+i+cnt).zfill(3) + ".txt")
                _ = shutil.move(save_directory + "/segment_kfold/masks/train/" + str(id+1+i+cnt).zfill(3) + ".png", save_directory + "/segment_kfold/masks/val/" + str(id+1+i+cnt).zfill(3) + ".png")
        cnt += len(subset)-1
    
    # Acceder a la capa de convolución inicial
    model = YOLO("yolov8n-seg.pt", task='segment')
    
    add_wandb_callback(model, enable_model_checkpointing=False)
    
    # Imprimimos el fold del que van a mostrarse los resultados
    print('--------------------------------')
    print(f"Model info:\n\t- Batch Size: {BATCH_SIZE}\n\t- GPUs on use: {torch.cuda.device_count()}")

    # Entrenamos el modelo, extrayendo los resultados y guardandolos en la variable result, y evaluamos en el conjunto de test.
    results = model.train(data = save_directory + "/segment_kfold/segment.yaml", 
                        pretrained = False, lr0 = 0.001, lrf = 0.01, 
                        epochs = num_epochs, batch = BATCH_SIZE, imgsz = window_shape, 
                        seed = 42, 
                        single_cls = True, 
                        workers = 8,  
                        mask_ratio = 1, close_mosaic = num_epochs//10, 
                        verbose = False, plots = True, 
                        project = "YOLOv8_PNeSegm", name = f'box3_cls05_dfl1_rect_noTL_fold{fold}', 
                        patience = 0, optimizer = "AdamW",
                        box = 3, cls = 0.5, dfl = 1,
                        rect = True,
                        save_dir = os.environ["STORE"]+ "/TFG/YOLOv8")

    # testloader = torch.utils.data.DataLoader(
    #                         dataset_test,
    #                         batch_size=BATCH_SIZE, num_workers=8, shuffle=False, persistent_workers=True)
    
    # Creamos un nuevo entrenador con una sola GPU para la fase de prueba
    # trainer_test = L.Trainer(devices = 1, strategy='auto', max_epochs=num_epochs, accelerator='cuda', log_every_n_steps=1, logger=logger_wandb, callbacks=callbacks)
    # trainer_test.test(model, testloader)

    wandb.finish()
    
    del model

    # Nuevo código para mover archivos de vuelta a las carpetas de entrenamiento
    val_folders = ['images', 'labels', 'masks']
    for folder in val_folders:
        val_path = os.path.join(save_directory, "segment_kfold", folder, "val")
        train_path = os.path.join(save_directory, "segment_kfold", folder, "train")
        
        for file_name in os.listdir(val_path):
            shutil.move(os.path.join(val_path, file_name), os.path.join(train_path, file_name))
        
    torch.cuda.empty_cache()
    time.sleep(30)

[rank: 0] Seed set to 42


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/netapp2/Home_FT2/home/ulc/co/ela/TFG/test/yolo_segmentation/segment_kfold/images/train/037.png'

In [26]:
from pnebulae_torch.dataset import NebulaeDataset
from pnebulae_torch.preprocess import  CutValues
from pnebulae_torch.normalize import TypicalImageNorm
from pnebulae_torch.utils import DivideWindowsSubset
from sklearn.model_selection import KFold
from torchvision import transforms
from lightning.pytorch import seed_everything
import torch
import os
import pandas as pd
import shutil

os.environ["WANDB_API_KEY"] = "21924e6e134841c5c16842c4ac42fcbe5a66feb2"
ruta_logs_wandb = os.environ["STORE"] + "/TFG/logs_wandb/"

torch.set_float32_matmul_precision('high')

####### CONFIGURACIÓN ENTRENAMIENTO #######
BATCH_SIZE = 124
num_epochs = 1000
lr = 1e-4
window_shape = 512

k = 5
    
############# CARGA DATASET #############
transform_x = transforms.Compose([
                    CutValues(factor = 2),
                    TypicalImageNorm(factor = 1, substract=0),
                    transforms.ToTensor(),
                    ])

transform_y = transforms.Compose([
                    transforms.ToTensor()
                    ])

df_train = pd.read_csv(route+"/data_files_1c_train.csv")
dataset_train = NebulaeDataset(data_directory, masks_directory, df_train, transform = (transform_x, transform_y))

seed_everything(42, workers = True)

########## ENTRENAMIENTO MODELO ##########
# Definimos el K-fold Cross Validator
kfold = KFold(n_splits=k, shuffle=True, random_state = 42)

for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset_train)):
    
    if fold == 1:
        cnt = 0
        for id in range(len(dataset_train)):
            subset = torch.utils.data.Subset(dataset_train, [id])
            subset = DivideWindowsSubset(subset, window_shape = window_shape, fill_min = True)
            
            if id in val_ids:
                for i in range(len(subset)):
                    _ = shutil.move(save_directory + "/segment_kfold/images/train/" + str(id+1+i+cnt).zfill(3) + ".png", save_directory + "/segment_kfold/images/val/" + str(id+1+i+cnt).zfill(3) + ".png")
                    _ = shutil.move(save_directory + "/segment_kfold/labels/train/" + str(id+1+i+cnt).zfill(3) + ".txt", save_directory + "/segment_kfold/labels/val/" + str(id+1+i+cnt).zfill(3) + ".txt")
                    _ = shutil.move(save_directory + "/segment_kfold/masks/train/" + str(id+1+i+cnt).zfill(3) + ".png", save_directory + "/segment_kfold/masks/val/" + str(id+1+i+cnt).zfill(3) + ".png")
            cnt += len(subset)-1
        break

[rank: 0] Seed set to 42


In [28]:
import shutil
# Nuevo código para mover archivos de vuelta a las carpetas de entrenamiento
val_folders = ['images', 'labels', 'masks']
for folder in val_folders:
    val_path = os.path.join(save_directory, "segment_kfold", folder, "val")
    train_path = os.path.join(save_directory, "segment_kfold", folder, "train")
    
    for file_name in os.listdir(val_path):
        shutil.move(os.path.join(val_path, file_name), os.path.join(train_path, file_name))